In [1]:
import pandas as pd
from dbfread import DBF

In [2]:
data_ruta=pd.DataFrame(iter(DBF('rekap_sp2020.dbf')))

In [3]:
import geopandas as gpd
from glob import glob

In [4]:
ls_sls=glob('sls_*.gpkg')

In [5]:
ls_sls

['sls_5300.gpkg', 'sls_5100_v2.gpkg', 'sls_5200.gpkg']

In [6]:
data_sls=gpd.GeoDataFrame()
for i in ls_sls:
    data_sls_i=gpd.read_file(i)
    data_sls=data_sls_i.append(data_sls)
    data_sls_i=None

C:\Users\user\AppData\Local\Temp\ipykernel_19992\283178317.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sls=data_sls_i.append(data_sls)
C:\Users\user\AppData\Local\Temp\ipykernel_19992\283178317.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sls=data_sls_i.append(data_sls)
C:\Users\user\AppData\Local\Temp\ipykernel_19992\283178317.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sls=data_sls_i.append(data_sls)


In [7]:
data_ruta.head(5)

,KODE_SLS,TOTAL_KELU,TOTAL_PEND,TOTAL_KE_A,TOTAL_PE_A,TOTAL_KE_B,TOTAL_PE_B
0,1101010001000100,41,135,0,0,42,135
1,1101010001000200,34,133,0,0,35,133
2,1101010001999900,2,3,0,5,2,8
3,1101010002000100,87,329,0,0,88,329
4,1101010002000200,39,175,0,0,40,175


In [8]:
data_ruta['idsls']=data_ruta.KODE_SLS.apply(lambda y: str(y)[:14])

In [9]:
#calculate per sls
data_ruta_compiled=pd.pivot_table(data_ruta,index='idsls',values='TOTAL_PEND',aggfunc='sum').reset_index()

In [10]:
data_sls['idsls']=data_sls.idsls.astype(str)
data_peta_w_ruta=data_sls[['geometry','idsls']].merge(data_ruta_compiled,how='left').fillna(0)

In [11]:
data_peta_w_ruta['area_pop']=data_peta_w_ruta.to_crs('ESRI:53034').area

In [12]:
#open built-up
data_builtup=gpd.read_file('BUILTUP_515253_REV.gpkg').to_crs(4326).dissolve()
data_builtup=data_builtup[['geometry']]

In [13]:
data_builtupsls=gpd.overlay(data_builtup,data_peta_w_ruta,
                            how='intersection').explode().reset_index()[['idsls','TOTAL_PEND','area_pop','geometry']]

C:\Users\user\AppData\Local\Temp\ipykernel_19992\856798148.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 5219 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  data_builtupsls=gpd.overlay(data_builtup,data_peta_w_ruta,
C:\Users\user\AppData\Local\Temp\ipykernel_19992\856798148.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  how='intersection').explode().reset_index()[['idsls','TOTAL_PEND','area_pop','geometry']]


In [14]:
data_builtupsls['area_built']=data_builtupsls.to_crs('ESRI:53034').area

In [15]:
table_area=pd.pivot_table(data_builtupsls[['idsls','area_built']],
                          index='idsls',values='area_built',aggfunc='sum').reset_index()

In [16]:
table_area.columns=['idsls','area_built_total']

In [17]:
data_builtupsls=data_builtupsls.merge(table_area).assign(per_built=lambda y:y.area_built/y.area_built_total*100 )
data_builtupsls['pop_built']=data_builtupsls.per_built*data_builtupsls.TOTAL_PEND/100

In [18]:
data_builtupsls.to_file('sls_builtup_5000.gpkg',driver='GPKG')

In [ ]:
#generate S2GEN 1km 

from s2gen import s2gen
y_=data_builtupsls.total_bounds
kml = s2gen(y_[3],y_[1],y_[2],y_[0], 13)

In [ ]:
with open("5000.kml", "w") as f:
    f.write(kml)

In [ ]:
import fiona

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
grid_1km=gpd.read_file('5000.kml',driver='KML')

In [ ]:
data_builtupslss2=gpd.overlay(grid_1km,data_builtupsls,how='intersection')

In [ ]:
data_builtupslss2['area_built_after']=data_builtupslss2.to_crs('ESRI:53034').area

In [ ]:
data_builtupslss2['per_built_after']=data_builtupslss2.area_built_after/data_builtupslss2.area_built

In [ ]:
data_builtupslss2['pop_built_s2']=data_builtupslss2.pop_built*data_builtupslss2.per_built_after

In [ ]:
tabel_pop_built_s2=pd.pivot_table(data_builtupslss2,index='Name',
                                  values=['pop_built_s2','per_built_after'],aggfunc='sum').reset_index()

In [ ]:
grid_with_pop=grid_1km.merge(tabel_pop_built_s2,how='left').fillna(0)
data_gridpops2=gpd.sjoin(grid_with_pop,data_sls[['geometry']],how='inner',op='intersects')


In [ ]:
(data_gridpops2[['Name','geometry','pop_built_s2','per_built_after']]
 .dissolve(by='Name').to_file('pop_dens_s2_5000.gpkg',driver='GPKG'))